In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

# Import Dependencies

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from scipy import stats

import plotly.graph_objects as go
import plotly.express as px

from utils import calculate_drawdown, calculate_sharpe_ratio, nearcorr

# Contour plot of 2 signals IC weighting

In [ ]:
perfs = np.linspace(-0.2, 0.2, 100)
corrs = np.linspace(-0.2, 0.9, 100)
weights = np.full((100, 100), np.nan)
for i, p in enumerate(perfs):
    for j, c in enumerate(corrs):
        perf = np.array([[0.1, p]])
        corr = np.array([[1, c], [c, 1]])
        w = perf @ np.linalg.pinv(corr)
        w = w[0]
        weights[j, i] = w[1]


fig = go.Figure(data =
    go.Contour(
        z=weights,
        x=perfs, # horizontal axis
        y=corrs, # vertical axis
        contours=dict(size=0.05, start=-2, end=0.5, showlabels=True, labelfont=dict(color="white"))
    ))
fig.update_yaxes(title="Signal Correlation")
fig.update_xaxes(title="Signal B IC")
fig.update_layout(
    title="Two Signals (A,B) - Contour Plot of Weight Assigned to Signal B. Signal A IC Fixed at 0.1",
    height=600, width=900
)
fig.show()

## Define some helper functions

In [ ]:
def generate_ics(seed: int, n_signals: int = 10, n_samples: int = 365 * 5) -> pd.DataFrame:
    """Generate time-series of IC values for each signal."""
    np.random.seed(seed + 69)
    a = np.random.normal(0.5, 1, (n_signals, n_signals))
    m = a.T @ a
    d = np.zeros_like(m)
    np.fill_diagonal(d, np.sqrt(np.diag(m)))
    corr = np.linalg.inv(d) @ m @ np.linalg.inv(d)
    corr = np.clip(corr, -0.1, 0.7)
    np.fill_diagonal(corr, 1)
    corr = 0.5 * (corr + corr.T)
    corr = nearcorr(corr)

    np.random.seed(seed + 42)
    ic_stds = np.random.uniform(0.0005, 0.001, n_signals)
    covs = corr.copy()
    for i in range(covs.shape[0]):
        for j in range(covs.shape[1]):
            covs[i, j] = corr[i, j] * ic_stds[i] * ic_stds[j]

    covs = 0.5 * (covs + covs.T)
    np.random.seed(seed + 8008)
    ic_mus = np.random.uniform(0.0001, 0.001, n_signals)
    ics = stats.multivariate_t(df=10, loc=ic_mus, shape=covs).rvs(n_samples, random_state=seed + 8008135)
    ics = pd.DataFrame(
        ics,
        index=pd.date_range("2025-01-01", periods=n_samples, freq="D"),
        columns=[f"signal_{i}" for i in range(n_signals)]
    ).clip(-1, 1)
    return ics

In [ ]:
def generate_signal_returns_from_ics(ics: pd.DataFrame, seed: int):
    """Generate signal returns correlated to IC's."""
    mu = ics.mean()
    sd = ics.std()
    z1 = (ics - mu) / sd
    rho = 0.95
    z2 = pd.DataFrame(stats.t(df=10, loc=0, scale=1).rvs(z1.shape, random_state=i + 999), columns=z1.columns, index=z1.index)
    signal_rets = rho * z1 + np.sqrt(1 - rho**2) * z2
    signal_rets = signal_rets * 10 * sd + mu
    return signal_rets

In [ ]:
def _rescale_weights(weights: pd.DataFrame) -> pd.DataFrame:
    """Re-scale weights so they are positive and sum to 1."""
    weights = weights.sub(weights.mean(axis=1), axis=0)
    weights = weights.div(weights.abs().sum(axis=1), axis=0)
    weights = 0.5 + weights
    weights = weights.div(weights.abs().sum(axis=1), axis=0)
    return weights


def get_ic_corr_weight(ics: pd.DataFrame) -> pd.DataFrame:
    """Get Type 1 weights."""
    weights = []
    idx = ics.index[28:]
    for dt in idx:
        curr_ics = ics.loc[dt - timedelta(days=365): dt]
        mu = curr_ics.mean().values.reshape(1, -1)
        icorr = np.linalg.pinv(curr_ics.corr().values)
        w = mu @ icorr
        # apply timedelta to avoid lookahead bias
        weights.append(
            pd.DataFrame(w, columns=curr_ics.columns, index=[dt + timedelta(days=1)])
        )

    weights = pd.concat(weights, axis=0)
    weights = _rescale_weights(weights=weights)
    return weights


def get_perf_corr_weight(perf_metric: pd.DataFrame) -> pd.DataFrame:
    """Get Type 2 weights."""
    weights = []
    idx = perf_metric.index[28:]
    for dt in idx:
        curr_perf = perf_metric.loc[dt - timedelta(days=365): dt]
        mu = curr_perf.iloc[-1].values.reshape(1, -1)
        icorr = np.linalg.pinv(curr_perf.corr().values)
        w = mu @ icorr
        # apply timedelta to avoid lookahead bias
        weights.append(
            pd.DataFrame(w, columns=curr_perf.columns, index=[dt + timedelta(days=1)])
        )

    weights = pd.concat(weights, axis=0)
    weights = _rescale_weights(weights=weights)
    return weights


def get_perf_corr_returns_weight(perf_metric: pd.DataFrame, signal_returns: pd.DataFrame) -> pd.DataFrame:
    """Get Type 3 weights."""
    weights = []
    idx = perf_metric.index[28:]
    for dt in idx:
        curr_signal_rets = signal_returns.loc[dt - timedelta(days=365): dt]
        mu = perf_metric.loc[dt].values.reshape(1, -1)
        icorr = np.linalg.pinv(curr_signal_rets.corr().values)
        w = mu @ icorr
        # apply timedelta to avoid lookahead bias
        weights.append(
            pd.DataFrame(w, columns=curr_signal_rets.columns, index=[dt + timedelta(days=1)])
        )

    weights = pd.concat(weights, axis=0)
    weights = _rescale_weights(weights=weights)
    return weights

# Run the full simulation.

In [ ]:
n_signals = 10
n_sims = 10_000
metrics = []
for i in range(1, n_sims):
    if i % 100 == 0:
        print(f"On iter {i}")
    try:
        ics = generate_ics(seed=i, n_signals=n_signals)
        sig_ret = generate_signal_returns_from_ics(ics=ics, seed=i * 4)

        rol = sig_ret.rolling(min_periods=28, window=365)
        rol_sharpes = np.sqrt(365) * rol.mean() / rol.std()

        rol_sharpes_scaled = rol_sharpes.copy()
        rol_sharpes_scaled = rol_sharpes_scaled.sub(rol_sharpes_scaled.mean(axis=1), axis=0)
        rol_sharpes_scaled = rol_sharpes_scaled.div(rol_sharpes_scaled.abs().sum(axis=1), axis=0)
        rol_sharpes_scaled = 0.5 + rol_sharpes_scaled

        weights_type1 = get_ic_corr_weight(ics=ics)
        weights_type2 = get_perf_corr_weight(perf_metric=rol_sharpes)
        weights_type3 = get_perf_corr_returns_weight(perf_metric=rol_sharpes_scaled, signal_returns=sig_ret)
        weights_type4 = rol_sharpes_scaled.div(rol_sharpes_scaled.sum(axis=1), axis=0)
        weights_type4 = weights_type4.shift(1)

        # calculate returns
        port_ret_weighted1 = (sig_ret * weights_type1).dropna().sum(axis=1)
        port_ret_weighted2 = (sig_ret * weights_type2).dropna().sum(axis=1)
        port_ret_weighted3 = (sig_ret * weights_type3).dropna().sum(axis=1)
        port_ret_weighted4 = (sig_ret * weights_type4).dropna().sum(axis=1)
        pr_eq = (sig_ret / n_signals).sum(axis=1)
        rnd_w = pd.DataFrame(
            np.random.uniform(0.1, 1, sig_ret.shape),
            columns=sig_ret.columns,
            index=sig_ret.index
        )
        rnd_w = rnd_w.sub(rnd_w.mean(axis=1), axis=0)
        rnd_w = rnd_w.div(rnd_w.abs().sum(axis=1), axis=0)
        rnd_w = 0.5 + rnd_w
        rnd_w = rnd_w.div(rnd_w.abs().sum(axis=1), axis=0)
        pr_rnd = (rnd_w * sig_ret).sum(axis=1)

        # calculate metrics
        dd_w1 = calculate_drawdown(port_ret_weighted1).min() * 100
        sharpe_w1 = calculate_sharpe_ratio(returns=port_ret_weighted1, scale=365, geometric=True)

        dd_w2 = calculate_drawdown(port_ret_weighted2).min() * 100
        sharpe_w2 = calculate_sharpe_ratio(returns=port_ret_weighted2, scale=365, geometric=True)

        dd_w3 = calculate_drawdown(port_ret_weighted3).min() * 100
        sharpe_w3 = calculate_sharpe_ratio(returns=port_ret_weighted3, scale=365, geometric=True)

        dd_w4 = calculate_drawdown(port_ret_weighted4).min() * 100
        sharpe_w4 = calculate_sharpe_ratio(returns=port_ret_weighted4, scale=365, geometric=True)


        dd_eq = calculate_drawdown(pr_eq).min() * 100
        sharpe_eq = calculate_sharpe_ratio(returns=pr_eq, scale=365, geometric=True)

        dd_rnd = calculate_drawdown(pr_rnd).min() * 100
        sharpe_rnd = calculate_sharpe_ratio(returns=pr_rnd, scale=365, geometric=True)

        metrics.append(
            pd.DataFrame(
                {
                    "Type1 - Sharpe": [sharpe_w1],
                    "Type2 - Sharpe": [sharpe_w2],
                    "Type3 - Sharpe": [sharpe_w3],
                    "Type4 - Sharpe": [sharpe_w4],

                    "Equal - Sharpe": [sharpe_eq],
                    "Random - Sharpe": [sharpe_rnd],

                    "Type1 - MDD (%)": [dd_w1],
                    "Type2 - MDD (%)": [dd_w2],
                    "Type3 - MDD (%)": [dd_w3],
                    "Type4 - MDD (%)": [dd_w4],

                    "Equal - MDD (%)": [dd_eq],
                    "Random - MDD (%)": [dd_rnd],
                }
            )
        )
    except Exception as err:
        print(f"Got err {err}")
    
metrics = pd.concat(metrics, axis=0).reset_index(drop=True)

In [ ]:
x = metrics.loc[:, [c for c in metrics.columns if "Sharpe" in c]]
x.columns = [c.split(" - ")[0] for c in x.columns]

fig = px.box(x)
fig.update_xaxes(title="Weighting Scheme")
fig.update_yaxes(title="Sharpe")
fig.update_layout(title="Sharpe Values of Different Weighting Schemes", width=700)
fig.show()

In [ ]:
x = metrics.loc[:, [c for c in metrics.columns if "DD" in c]]
x.columns = [c.split(" - ")[0] for c in x.columns]

fig = px.box(x)
fig.update_xaxes(title="Weighting Scheme")
fig.update_yaxes(title="MDD (%)")
fig.update_layout(title="Max Dradown Values of Different Weighting Schemes", width=700)
fig.show()